# EDA and Modeling of Medical Data


<b>Problem Statement:</b> We are tasked with creating an automated system to estimate the annual medical charges for new customers, using information such as their age, sex, BMI, children, smoking habits and region of residence. Estimates from the system will be used to determine the annual insurance premium offered to the customer.

<ins>What is a similar manufacturing problem statement?</ins>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Read data from file

df= pd.read_csv('insurance.csv')

In [ ]:
# Check raw data
df.head()

In [ ]:
# Check size of dataframe

df.shape

In [ ]:
# Check column names

columns=df.columns

print("Columns\n----------")
for each in columns:
    print(each)

In [ ]:
# Check data type in each column
# Check for missing values

df.info()

In [ ]:
# Another way to explicitly check for missing values

print("Number of missing values found")
df.isnull().sum()


#### No null values found. What next?

In [ ]:
# Check for duplicates

print(f"Number of duplicates found: {df.duplicated().sum()}")

#### We found a single duplicate, what should we do with it?

In [ ]:
# Remove the duplicated row

print(f"Size before removal: {df.shape}")

# Drop the duplicate row
df=df.drop_duplicates()

print(f"Size after removal: {df.shape}")

In [ ]:
df.describe()

In [ ]:
# Lets look at the value counts

for i in columns:
  print (i)
  print(df[i].value_counts())
  print('--------------------------------')

### Not the easiest way to visualize the data. How might we visualize it? 

In [ ]:
sns.pairplot(df);

# Sex and smoker excluded in this plot

### What do you see in the data?


We can partition the data ourselves for better vizualization. Let's apply labels based on the USA NIH Body Mass Index categories. https://www.nhlbi.nih.gov/health/educational/lose_wt/BMI/bmicalc.htm

Underweight: <18.5
Normal weight: 18.5-24.9
Overweight: 25-29.9
Obesity: >30

In [ ]:
# Return a label (str) based on input BMI value
def bmi_to_cat(bmi:float) -> str:
  if bmi<18.5:
     return 'A-Underweight'
  elif 18.5<=bmi<25:
     return 'B-Normal weight'
  elif 25<=bmi<30:
     return 'C-Overweight'
  elif 30<=bmi:
     return 'D-Obese'

# Create new column at position 3 and using function to apply a label to each column
df.insert(3,'bmi_cat',df['bmi'].apply(bmi_to_cat))
df.head()

In [ ]:
fig=px.scatter(df,
               x='bmi',
               y='age',
               color='bmi_cat')

# reverse order of legend
fig.update_layout(legend_traceorder="reversed")
fig.show()

#### What do you takeaway from this plot?

In [ ]:
# plot the counts of BMI category by sex

fig=px.histogram(df,x='bmi_cat',color='sex', text_auto=True)
fig.show()

#### Before we go any further, remember back to the distribution of charges? Let's look at that before we start to think about relationships between charge and other features.

In [ ]:
# Histogram of charges as given
fig = px.histogram(df,
    x='charges', # plot charges on x
    nbins=32,
    histfunc = 'count', # bin by count
    color_discrete_sequence = ['#F56600'],
    marginal="rug" # add a margin plot showing the distribution
)

fig.show()

#### Is this a good distribution for statistical analysis?

There is a clear right skew to the data distribution

What can we do about this?

- <b>Do nothing:</b> Many statistical tests, including t tests, ANOVAs, and linear regressions, aren’t very sensitive to skewed data. Especially if the skew is mild or moderate, it may be best to ignore it.

- <b>Use a different model:</b> You may want to choose a model that doesn’t assume a normal distribution. Non-parametric tests or generalized linear models could be more appropriate for your data.

- <b>Transform the variable:</b> Another option is to transform a skewed variable so that it’s less skewed. “Transform” means to apply the same function to all the observations of a variable.

<img src="Transformation by Skew level and side.png" alt="Transformation by Skew level and side"/>

Image source: https://www.scribbr.com/statistics/skewness/

In [ ]:
import plotly.graph_objs as go
from plotly import subplots
from plotly.offline import iplot

# Determine the distribution of charge
charge_dist = df["charges"].values # charges as given
logcharge = np.log(df["charges"]) # natural log of charge value

# Histogram of charges as given
trace0 = go.Histogram(
    x=charge_dist,
    histfunc='count',
    name="Charges Distribution",
    marker = dict(
        color = '#F56600',
    )
)

# Histogram of natural log of charges
trace1 = go.Histogram(
    x=logcharge,
    histfunc='count',
    name="Charges Distribution using Log",
    marker = dict(
        color = '#522D80',
    )
)

# figure to hold both histograms
fig = subplots.make_subplots(rows=2, cols=1,
                          subplot_titles=('Charge Distribution','Log Charge Distribution'),
                         print_grid=False)


# add two histograms to figure
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)

# display figure
fig['layout'].update(showlegend=True, 
                     title='Charge Distribution by value and log(value)', 
                     bargap=0.05
                    )
iplot(fig, filename='custom-sized-subplot-with-subplot-titles')

#### Now let's dig into the features. We haven't looked at the smoker feature yet.

In [ ]:
# Table of values by sex and smoking status. 
# Also include the counts along rows and columns
pd.crosstab(df.sex,df.smoker,margins=True)

In [ ]:
# Strip chart (1-D jittered scatter plot) of sex vs charges and split based on smoker

fig=px.strip(df,x='sex',y='charges',color='smoker')
fig.show()

### Observations?

Observations: For most customers, the annual medical charges are below 15,000 dollars. But there are outliers who have higher medical expenses, possibly due to some other reasons. However, there is a significant difference in medical expenses between smokers and non-smokers.

#### Let's look at the breakdown of sex, smoker, and BMI by count.

In [ ]:
# pivot table breakdown of sex>smoker and BMI counts
pd.crosstab([df.sex,df.smoker],df.bmi_cat,margins=True)

In [ ]:
# Group the data by sex, smoker, and BMI category then find the mean of the charge

a=df.groupby(['sex','smoker','bmi_cat'])['charges'].mean().reset_index()
a.head(20) 

# NOTE: there are only 16 rows so we can use any number larger than the expected number to print all

In [ ]:
# Using the grouped data
# plot charges by sex and BMI category
# Group bars for smoker

fig=px.bar(a,x='sex',y='charges',color='bmi_cat',barmode='group', text_auto=True)
fig.show()

In [ ]:
# plot charges by sex, smoker, and BMI category

fig=px.histogram(df,x='bmi_cat',color='sex',barmode='group',facet_row='smoker', text_auto=True)
fig.show()

We can observe that the average charges for non-smoking males & females of different bmi categories, is <\\$10k. Both sexes who are smokers as well as any form of obese, have average charges as high as \\$43k. Male underweight had the lowest category average cost of smokers with an average of \\$13k

We haven't looked at region yet.

In [ ]:
# pivot table of BMI category by region

pd.crosstab(df.region,df.bmi_cat,margins=True)

In [ ]:
# Charges vs region and BMI category
fig=px.strip(df,x='region',y='charges',color='bmi_cat')
# reverse order of legend
fig.update_layout(legend_traceorder="reversed")
fig.show()

In [ ]:
# pivot table of smoker status by region
pd.crosstab(df.region,df.smoker,margins=True)

In [ ]:
# Plot the pivot table of smoker status by region
fig=px.histogram(df, x='region', facet_row='smoker', text_auto=True)
fig.show()

In [ ]:
# plot charges by region and smoker

fig=px.strip(df,x='region',y='charges',color='smoker')
fig.show()

### Observations?

In [ ]:
# scatter plot charges by age and color by smoker
fig=px.scatter(df,x='age',y='charges',color='smoker')
fig.show()

### Observations?

Medical charges increase with age

We can see three clusters of points, each of which seems to form a line with an increasing slope:

- The first cluster consists of non-smokers who have relatively low medical charges compared to others.
- The second cluster contains a mix of smokers and non-smokers. It's possible that these are actually two distinct but overlapping clusters: "non-smokers with other medical issues" and "smokers without major medical issues".
- The final cluster consists of smokers with major medical issues that are possibly related to or worsened by smoking.

In [ ]:
# Plot the empirical cumulative distribution plot of charges by smoker

# The empirical distribution function is an estimate of the cumulative 
# distribution function that generated the points in the sample. It 
# converges with probability 1 to that underlying distribution. Helps 
# answer the question, how likely are we to see a value below a certain point.

# more info on ecdf https://en.wikipedia.org/wiki/Empirical_distribution_function

fig=px.ecdf(df,x='charges',color='smoker')
fig.show()



### Observations?

2/3 of non-smokers have charges less than \\$10k.

2/3 of smokers have charges less than \\$40k.

In [ ]:
# Plot the empirical cumulative distribution plot of charges by sex

fig=px.ecdf(df,x='charges',color='sex')
fig.show()

In [ ]:
# Plot the empirical cumulative distribution plot of charges by region

fig=px.ecdf(df,x='charges',color='region')
fig.show()

In [ ]:
# Scatter plot of charges vs BMI, color by smoker

fig=px.scatter(df,x='bmi',y='charges',color='smoker')
fig.show()

### Observations?

It appears that for non-smokers, an increase in BMI doesn't seem to be related to an increase in medical charges. However, medical charges seem to be significantly higher for smokers with a BMI greater than 30.

Let's try another way to visualize the data with a 3D plot.

In [ ]:
# 3D Scatter plot of charges vs BMI and age, color by smoker

fig=px.scatter_3d(df,x='age',y='bmi',z='charges',color='smoker')
fig.update_traces(marker_size=5,marker_opacity=0.5)
fig.show()

***
## Preparing the data for modeling

- Clean dataset as needed (here it is already very clean!)
- Encode categorical variables
- Check for correlation in features
- Scale columns as needed

In [ ]:
# Remind ourselves what we are working with
df.head()

In [ ]:
# Drop our generated column BMI category

df.drop(['bmi_cat'],axis=1,inplace=True)
df.head()

We will need to encode the columns sex, smoker, and region.

In [ ]:
columns_to_encode=['sex','smoker','region']

df_encoded=pd.get_dummies(df,columns=columns_to_encode)
df_encoded.head()

In [ ]:
# lets take a look at the feature correlations

plt.figure(figsize=(10,10))
sns.heatmap(df_encoded.corr(),linewidths=0.5,annot=True)

### Observations?

The newly created sex_male and sex_female columns are perfectly inversely correlated in our data. 

The newly created smoker_yes and smoker_female columns are perfectly inversely correlated in our data. 

region_southeast and bmi have a positive correlation

For the feature of interest for prediction
- age and BMI have a positive correlation
- smoker_no and smoker_yes have a strong negative and positive correlation, respectively

### K-1 Dummy Encoding
Converts a categorial variable with k number of labels into k-1 binary variables that are better suited for ML applications than strings.

<img src="k-1_dummy encoding.png" alt="k-1_dummy encoding.png">

Why is it called k-1?

Let's try encoding again, this time using K-1.

In [ ]:
columns_to_k1_encode=['sex','smoker']
columns_to_encode=['region']

df_encoded=pd.get_dummies(df,columns=columns_to_k1_encode, drop_first=True)
df_encoded=pd.get_dummies(df_encoded,columns=columns_to_encode, drop_first=False)
df_encoded.head()

In [ ]:
# Pairplot of all features of encoded data

sns.pairplot(df_encoded);

In [ ]:
# lets take a look at the feature correlations

plt.figure(figsize=(10,10))
sns.heatmap(df_encoded.corr(),linewidths=0.5,annot=True);

***
## Modeling
### Simple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# build our feature and response dataframe and series
x = df_encoded.drop(['charges'], axis = 1)
y = df_encoded.charges

In [ ]:
# Split the data 
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.4, random_state = 7)

# define our model
lr = LinearRegression()

# fit the model to the training data
lr.fit(x_train,y_train)

# predict the 
y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print("Training Performance")
print(f"R2 = {lr.score(x_train, y_train):.2f}")

print("\nTesting Performance")
print(f"R2 = {lr.score(x_test,y_test):.2f}")

In [ ]:
# Check other performance metrics
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_test_pred))
r2 = np.abs(r2_score(y_test,y_test_pred))

print(f'MSE test: {mse:.2f}')
print(f'RMSE test: {rmse:.2f}')

Not bad performance from just throwing the data into a model! Lets take a look at a plot of the prediction vs actual value of charges.

In [ ]:
# plot the value of predicted vs actual
scale = 500

fig=px.scatter(x=y_test,y=y_test_pred,labels={'x':'Actual','y':'Predicted'}, width=scale, height=scale)
# draw a line behind the data points
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig.show()

It is ok, but there is a spread of data points that don't fall on the expected diagonal.

Let's check linearity using OLS.


In [ ]:
# spot check regression
scale = 500
px.scatter(df,'age','charges',trendline='ols',trendline_color_override='black', width=scale, height=scale)

In [ ]:
# Examine the weights of each feature
print(lr.coef_)

weights_df=pd.DataFrame({'features':np.append(x.columns,1),'weights':np.append(lr.coef_,lr.intercept_)})
weights_df.sort_values(by='weights', ascending=False)

While it seems like "smoker_yes", "region_northeast", and "bmi" have a higher weight than age, keep in mind that their range of values columns only take the values 0 and 1, while that of bmi ranges from 15 to 53.

Because different columns have different ranges, we run into two issues:

We can't compare the weights of different columns to identify which features are important. A column with a larger range of inputs may disproportionately affect the loss and dominate the optimization process. For this reason, it's common practice to scale (or standardize) the values in numerical columns. We can apply scaling using the StandardScaler class from scikit-learn

In [ ]:
# Scale the features using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

# numerical columns to scale
numerical_columns=['age','bmi','children']
scaler.fit(df_encoded[numerical_columns])

# scale numerical columns and remake into a new DataFrame
scaled_inputs=pd.DataFrame(scaler.transform(df_encoded[numerical_columns]), columns=numerical_columns)

# print(scaled_inputs)

# drop the unscaled columns and put the numerical columns into a new DataFrame
df_encoded_scaled = df_encoded.drop(numerical_columns, axis=1)

# print(df_encoded_scaled)

# concatenate the scaled DataFrame and the numerical data
df_encoded_scaled = pd.concat([df_encoded_scaled, scaled_inputs], axis=1)
df_encoded_scaled=df_encoded_scaled.dropna()


x = df_encoded_scaled.drop(['charges'], axis=1)
y = df_encoded_scaled.charges

# split dataset
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.4, random_state = 7)

# Train model
model = lr.fit(x_train, y_train)

y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print("Training Performance")
print(f"R2 = {lr.score(x_train, y_train):.2f}")

print("\nTesting Performance")
print(f"R2 = {lr.score(x_test,y_test):.2f}")

# Check other performance metrics
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_test_pred))
r2 = np.abs(r2_score(y_test,y_test_pred))

print('\n')
print(f'MSE test: {mse:.2f}')
print(f'RMSE test: {rmse:.2f}')

Scaling affected the model performance, which is expected. Let's look at the weights again.

In [ ]:
# Examine the weights of each feature
weights_df=pd.DataFrame({'features':np.append(x.columns,1),'weights':np.append(lr.coef_,lr.intercept_)})
weights_df.sort_values(by='weights', ascending=False)

### Observations?

Let's try polynomial features.

In [ ]:
# build our feature and response dataframe and series
x = df_encoded.drop(['charges'], axis = 1)
y = df_encoded.charges

quad = PolynomialFeatures (degree = 2)
x_quad = quad.fit_transform(x) 

# Example, if an input sample is two dimensional and of the form [a, b], 
# the degree-2 polynomial features are [1, a, b, a^2, ab, b^2]

# NOTE: Be aware that the number of features in the output array scales polynomially 
# in the number of features of the input array, and exponentially in the degree. 
# High degrees can cause overfitting. 

x_train,x_test,y_train,y_test = train_test_split(x_quad,y, random_state = 0)

plr = LinearRegression().fit(x_train,y_train)

y_train_pred = plr.predict(x_train)
y_test_pred = plr.predict(x_test)

print("Training Performance")
print(f"R2 = {plr.score(x_train, y_train):.2f}")

print("\nTesting Performance")
print(f"R2 = {plr.score(x_test,y_test):.2f}")

# Check other performance metrics
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_test_pred))
r2 = np.abs(r2_score(y_test,y_test_pred))

print('\n')
print(f'MSE test: {mse:.2f}')
print(f'RMSE test: {rmse:.2f}')

In [ ]:
# plot the value of predicted vs actual for polynomial features
scale = 500

fig=px.scatter(x=y_test,y=y_test_pred,labels={'x':'Actual','y':'Predicted'}, width=scale, height=scale)
# draw a line behind the data points
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig.show()

Again, good performance!

Let's try a random forest regression. 

In [ ]:
# build our feature and response dataframe and series
x = df_encoded.drop(['charges'], axis = 1)
y = df_encoded.charges

# split the data 
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.4, random_state = 7)

# define our model
clf = RandomForestRegressor(n_estimators = 100,
                              criterion = 'friedman_mse',
                              random_state = 10,
                              n_jobs = -1)

clf.fit(x_train,y_train)
forest_train_pred = clf.predict(x_train)
forest_test_pred = clf.predict(x_test)

print(f'MSE train data: {mean_squared_error(y_train,forest_train_pred):.3f}') 
print(f'MSE test data: {mean_squared_error(y_test,forest_test_pred):.3f}')
print("\n")
print(f'R2 train data: {r2_score(y_train,forest_train_pred):.2f}')
print(f'R2 test data: {r2_score(y_test,forest_test_pred):.2f}')

It works well but R2 is not significantly better than the polynomial feature regression.

In [ ]:
# plot the value of predicted vs actual
scale = 500

fig=px.scatter(x=y_test,y=forest_test_pred,labels={'x':'Actual','y':'Predicted'}, width=scale, height=scale)
# draw a line behind the data points
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig.show()

#### Obervations? 

#### How might we improve this plot?

In [ ]:
# Plot tailings vs predicted values
# How far away from the actual value was the model?

plt.figure(figsize=(8,6))

plt.scatter(forest_train_pred,forest_train_pred - y_train,
          c = '#522D80', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(forest_test_pred,forest_test_pred - y_test,
          c = '#F56600', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["features"] = df_encoded.columns

vif_data["VIF"] = [variance_inflation_factor(df_encoded.values, i)
                   for i in range(len(df_encoded.columns))]

# Print the sorted VIF data output from high to low value
vif_data.sort_values(ascending=False, by='VIF')

### Observations?